# New RUN Key Pointing to Suspicious Folder
Detects suspicious new RUN key element pointing to an executable in a suspicious folder

## Rule Content
```
- title: New RUN Key Pointing to Suspicious Folder
  id: 02ee49e2-e294-4d0f-9278-f5b3212fc588
  status: experimental
  description: Detects suspicious new RUN key element pointing to an executable in
    a suspicious folder
  references:
  - https://www.fireeye.com/blog/threat-research/2018/08/fin7-pursuing-an-enigmatic-and-evasive-global-criminal-operation.html
  author: Florian Roth, Markus Neis
  tags:
  - attack.persistence
  - attack.t1060
  date: 2018/25/08
  modified: 2019/10/01
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection:
      EventID: 13
      TargetObject:
      - '*\SOFTWARE\Microsoft\Windows\CurrentVersion\Run\\*'
      - '*\SOFTWARE\Microsoft\Windows\CurrentVersion\RunOnce\\*'
      Details:
      - '*C:\Windows\Temp\\*'
      - '*\AppData\\*'
      - '%AppData%\\*'
      - '*C:\$Recycle.bin\\*'
      - '*C:\Temp\\*'
      - '*C:\Users\Public\\*'
      - '%Public%\\*'
      - '*C:\Users\Default\\*'
      - '*C:\Users\Desktop\\*'
      - wscript*
      - cscript*
    condition: selection
  fields:
  - Image
  falsepositives:
  - Software with rare behaviour
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"13" AND registry_key_path.keyword:(*\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\Run\\* OR *\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\RunOnce\\*) AND registry_key_value.keyword:(*C\:\\Windows\\Temp\\* OR *\\AppData\\* OR %AppData%\\* OR *C\:\\$Recycle.bin\\* OR *C\:\\Temp\\* OR *C\:\\Users\\Public\\* OR %Public%\\* OR *C\:\\Users\\Default\\* OR *C\:\\Users\\Desktop\\* OR wscript* OR cscript*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()